# Getting and preparing the data



To demonstrate the post processing algorithm we use the "Adult Data Set" from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/adult). The task there is to predict whether a person makes more or less than $50k based on just a few attributes such as age, sex, race, education, occupation, etc.
For the purposes of this notebook, we can interpret this as a loan decision problem. The label indicates whether or not each individual repaid a loan in the past. We will use the data to train a model to predict whether previously unseen individuals will repay a loan or not. The assumption is that the model predictions are used to decide whether an individual should be offered a loan.

To start, let's download the dataset using `shap`

In [ ]:
import pandas as pd
import numpy as np
import shap

X, y = shap.datasets.adult()
X.head()

Since we need to encode categorical attributes properly, there are a few additional steps to take. We'll also split the data into train and test sets. The attribute we're interested in for the purpose of this notebook is `Sex`. Female is encoded as 0 and male as 1. Note: our approach works even for more than two groups, but the dataset didn't provide that information.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

A = X["Sex"]
X = pd.get_dummies(X)

y = LabelEncoder().fit_transform(y)


X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(X, 
                                                                     y, 
                                                                     A,
                                                                     test_size = 0.2,
                                                                     random_state=0,
                                                                     stratify=y)

X_train = X_train.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)

# Create a fairness-unaware model

In [ ]:
from sklearn.linear_model import LogisticRegression

fairness_unaware_model = LogisticRegression(solver='liblinear')
fairness_unaware_model.fit(X_train, y_train)

def show_proportions(X, y_pred, y=None, description=None):
    male_indices = X.index[X.Sex == 1]
    female_indices = X.index[X.Sex == 0]

    males_earning_more_than_50k = sum(y_pred[male_indices])
    females_earning_more_than_50k = sum(y_pred[female_indices])
    print("\n" + description)
    print("P[loan approval | male] = {}".format(males_earning_more_than_50k/len(male_indices)))
    print("P[loan approval | female] = {}".format(females_earning_more_than_50k/len(female_indices)))
    
    if y is not None:
        positive_male_indices = X.index[(X.Sex == 1) & (y == 1)]
        negative_male_indices = X.index[(X.Sex == 1) & (y == 0)]
        positive_female_indices = X.index[(X.Sex == 0) & (y == 1)]
        negative_female_indices = X.index[(X.Sex == 0) & (y == 0)]
        print("P[loan approval | male, loan repaid] = {}".format(sum(y_pred[positive_male_indices])/len(positive_male_indices)))
        print("P[loan approval | female, loan repaid] = {}".format(sum(y_pred[positive_female_indices])/len(positive_female_indices)))
        print("P[loan approval | male, loan not repaid] = {}".format(sum(y_pred[negative_male_indices])/len(negative_male_indices)))
        print("P[loan approval | female, loan not repaid] = {}".format(sum(y_pred[negative_female_indices])/len(negative_female_indices)))
    
show_proportions(X_train, y_train, description="original training data:")
show_proportions(X_train, fairness_unaware_model.predict(X_train), y_train, description="fairness-unaware prediction on training data:")
show_proportions(X_test, y_test, description="original test data:")
show_proportions(X_test, fairness_unaware_model.predict(X_test), y_test, description="fairness-unaware prediction on test data:")

We notice a stark contrast in the predictions with males being a lot more likely to be predicted to get approved, similar to the original training data. However, there's even a disparity between the subgroup of males and females that are approved with 54.7% of males predicted to get approved, and only 42.4% of females. When considering only the samples labeled with "loan not repaid" males (9.8%) are more than five times as likely as females (1.4%) to be predicted to get approved. The test data shows a similar disparity.

In [ ]:
from azureml.contrib.explain.model.visualize import FairnessDashboard

FairnessDashboard([fairness_unaware_model,fairness_unaware_model,fairness_unaware_model], X_test, y_test.tolist(), pd.DataFrame(A_test).values.tolist(), True, list(X_test.columns), [0, 1], ["Sex"])

# Post-processing the model to get a fair model

The idea behind post-processing is to alter the output of the fairness-unaware model to achieve fairness. The post-processing algorithm requires three input arguments:
- the matrix of samples X
- the vector of predictions y from the fairness-unaware model 
- the vector of protected attribute values A

The goal is to make the output fair with respect to a disparity metric. Our post-processing algorithm uses one of
- Demographic Parity (DP): $P[h(X)=\hat{y} | A=a] = P[h(X)=\hat{y}] \qquad \forall a, \hat{y}$
- Equalized Odds (EO): $P[h(X)=\hat{y} | A=a, Y=y] = P[h(X)=\hat{y}|Y=y] \qquad \forall a, \hat{y}$

where $h(X)$ is the prediction based on the input $X$, $\hat{y}$ and $y$ are labels, and $a$ is a protected attribute value. In our example, we'd expect the post-processed model with DP to be balanced between sexes. EO does not make the same guarantees. Instead, it ensures that the parity between the subgroups of each sex with label 1 in the training set, and parity between the subgroups of each sex with label 0 in the training set. Applied to our scenario, this means that men and women with who have repaid their loan in the past are equally likely to be approved for a new loan (and therefore also equally likely to be rejected). Similarly, there is parity between men and women who have not repaid a loan, but we have no parity between the groups with different training labels. In mathematical terms:

$$
P[\text{loan approval} | \text{male, loan repaid}] = P[\text{loan approval} | \text{female, loan repaid}], \text{e.g. } 0.95\\
P[\text{loan approval} | \text{male, loan not repaid}] = P[\text{loan approval} | \text{female, loan not repaid}], \text{e.g. } 0.15
$$

but that also means that men (and women) of different subgroup based on training labels don't necessarily have parity:

$$
P[\text{loan approval} | \text{male, loan repaid}] = 0.95 \neq 0.15 = P[\text{loan approval} | \text{male, loan not repaid}]
$$

Assessing which disparity metric is indeed fair varies by application scenario.

In [ ]:
from fairlearn.post_processing import ROCCurveBasedPostProcessing
from copy import deepcopy

post_processed_model_DP = ROCCurveBasedPostProcessing(fairness_unaware_model=deepcopy(fairness_unaware_model), disparity_metric="DemographicParity", plot=True, seed=0)

post_processed_model_DP.fit(X_train, y_train, aux_data=X_train.Sex)

fairness_aware_predictions_DP_train = post_processed_model_DP.predict(X_train, group_data=X_train.Sex)
fairness_aware_predictions_DP_test = post_processed_model_DP.predict(X_test, group_data=X_test.Sex)

show_proportions(X_train, fairness_aware_predictions_DP_train, y_train, description="demographic parity with post-processed model on training data:")
show_proportions(X_test, fairness_aware_predictions_DP_test, y_test, description="demographic parity with post-processed model on test data:")

In [ ]:
from fairlearn.post_processing import ROCCurveBasedPostProcessing
from copy import deepcopy

post_processed_model_EO = ROCCurveBasedPostProcessing(fairness_unaware_model=deepcopy(fairness_unaware_model), disparity_metric="EqualizedOdds", plot=True, seed=0)

post_processed_model_EO.fit(X_train, y_train, aux_data=X_train.Sex)

fairness_aware_predictions_EO_train = post_processed_model_EO.predict(X_train, group_data=X_train.Sex)
fairness_aware_predictions_EO_test = post_processed_model_EO.predict(X_test, group_data=X_test.Sex)

show_proportions(X_train, fairness_aware_predictions_EO_train, y_train, description="equalized odds with post-processed model on training data:")
show_proportions(X_test, fairness_aware_predictions_EO_test, y_test, description="equalized odds with post-processed model on test data:")